# Analysis of electric car usage
## Research Question

- Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018.

## Bonus Questions (Optional)

- What is the most popular hour for returning cars?
- What station is the most popular? 
- -Overall?
- At the most popular picking hour?
- What postal code is the most popular for picking up Blue cars? Does the most popular station belong to that postal code?
- - Overall?
- -At the most popular picking hour?
- Do the results change if you consider Utilib and Utilib 1.4 instead of Blue cars? 

## 1.0 Importing libraries

In [1]:
# Importing the libraries we will need 

# Importing the pandas library
# 
import pandas as pd

# Importing the numpy library
#
import numpy as np

# Importing datetime library
#
import datetime as dt

## 1.1 Reading Dataset from CSV file

In [2]:
# Loading the dataset
#
url = 'http://bit.ly/autolib_dataset'
autolib_data = pd.read_csv(url)

## 1.2 Previewing the data set

In [3]:
# Getting to know more about the dataset by accessing its information
#
autolib_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Cars                 5000 non-null   int64 
 2   Bluecar counter      5000 non-null   int64 
 3   Utilib counter       5000 non-null   int64 
 4   Utilib 1.4 counter   5000 non-null   int64 
 5   Charge Slots         5000 non-null   int64 
 6   Charging Status      5000 non-null   object
 7   City                 5000 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   5000 non-null   object
 10  Kind                 5000 non-null   object
 11  Geo point            5000 non-null   object
 12  Postal code          5000 non-null   int64 
 13  Public name          5000 non-null   object
 14  Rental status        5000 non-null   object
 15  Scheduled at         47 non-null     object
 16  Slots 

## 1.3 Accessing information about dataset

In [4]:
# Previewing the firts five rows of taxi data
#
autolib_data.head()

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,1,operational,Paris,NaN,paris-eugeneoudine-51,STATION,"48.8250327, 2.3725162",75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23


## 1.4 Data cleaning


### 1.4.1) <font color="green">Validity</font>

 adding columns, Dropping unwated columns, merging columns to get desired attributings, and changing of datatypes of the various columns

In [5]:
# Add column seconds
#
autolib_data['sec'] = '00'

In [6]:
# Joining year day and month
# Converting year column from integer to string
#
autolib_data['year'] = autolib_data['year'].astype(str)

# Converting month column from integer to string
#
autolib_data['month'] = autolib_data['month'].astype(str)

#Converting day column from integer to string
#
autolib_data['day'] = autolib_data['day'].astype(str)
# Joining the year, day, and month column in the (YYYY-MM-DD) format
#
autolib_data['Date'] = autolib_data[['year', 'month', 'day']].agg('-'.join, axis = 1)

# Previewing the data
#
autolib_data.head()

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute,sec,Date
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43,00,2018-4-8
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24,00,2018-4-6
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14,00,2018-4-3
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,1,operational,Paris,NaN,paris-eugeneoudine-51,STATION,"48.8250327, 2.3725162",75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37,00,2018-4-4
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23,00,2018-4-8


In [7]:
# Joing the time data into one row
# Converting the hour column from integer to dtring
#
autolib_data['hour'] = autolib_data['hour'].astype(str)

# Converting the minute column from minute to time
#
autolib_data['minute'] = autolib_data['minute'].astype(str)

# Joining the minute and hour columns into a new column Time
#
autolib_data['Time'] = autolib_data[['hour', 'minute', 'sec']].agg(':'.join, axis = 1)


In [8]:
# Joining the date to time columns
#
autolib_data['datetime'] = autolib_data['Date'] + ' ' + autolib_data['Time'] 
autolib_data.head(1)

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute,sec,Date,Time,datetime
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43,00,2018-4-8,11:43:00,2018-4-8 11:43:00


joining the date to the time column to form datetime column

In [9]:
# Converting time and date clumns to datetime format
# Converting date to datetime data type
#
autolib_data['datetime'] = pd.to_datetime(autolib_data['datetime'])

Datetime column is changed from object to datetime format

In [10]:
# Dropping unwanted columns
# Dropping minute
#
autolib_data.drop('minute', inplace = True, axis = 'columns')

# Dropping sec
#
autolib_data.drop('sec', inplace = True, axis = 'columns')

# Dropping date
#
autolib_data.drop('Date', inplace = True, axis = 'columns')

# dropping time
#
autolib_data.drop('Time', inplace = True, axis = 'columns')

# Dropping cars, since similar to bluecar counter
#
autolib_data.drop('Cars', inplace = True, axis = 'columns')

# Previewing the data
#
autolib_data.head()

,Address,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,datetime
0,2 Avenue de Suffren,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,2018-04-08 11:43:00
1,145 Rue Raymond Losserand,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,2018-04-06 07:24:00
2,2 Avenue John Fitzgerald Kennedy,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,2018-04-03 20:14:00
3,51 Rue EugÃ¨ne OudinÃ©,3,1,0,1,operational,Paris,NaN,paris-eugeneoudine-51,STATION,"48.8250327, 2.3725162",75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,2018-04-04 04:37:00
4,6 avenue de la Porte de Champerret,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,2018-04-08 17:23:00


- The sec and min columns are dropped since thier use becomes redudant once merged in the HH:MM:SS format. 
- The cars column is dropped since it is similar to theblue car counter column. 
- The hour is maintained since it is needed for subsequent analysis

### 1.4.2) <font color="green">Accuracy

</font>

Investigating columns for errors

In [11]:
# querying column information
#
autolib_data.columns

Index(['Address', 'Bluecar counter', 'Utilib counter', 'Utilib 1.4 counter',
       'Charge Slots', 'Charging Status', 'City', 'Displayed comment', 'ID',
       'Kind', 'Geo point', 'Postal code', 'Public name', 'Rental status',
       'Scheduled at', 'Slots', 'Station type', 'Status',
       'Subscription status', 'year', 'month', 'day', 'hour', 'datetime'],
      dtype='object')

All columns seem accurate and have no wide spaces

### 1.4.3) <font color="green">Completeness</font>

Investigating null values in the autolib dataset

In [12]:
# Ascertain null values per column
# Find null values in boolean
#
autolib_data.isnull()

# Confirm if there are any null values in bolean
#
autolib_data.isnull().any()
# Find the total number of null values in dataset
#
autolib_data.isnull().sum().sum()

# Overview of null values per column
#
autolib_data.isnull().describe()

,Address,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,datetime
count,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000
unique,1,1,1,1,1,1,1,2,1,1,1,1,1,1,2,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
freq,5000,5000,5000,5000,5000,5000,5000,4889,5000,5000,5000,5000,5000,5000,4953,5000,5000,5000,5000,5000,5000,5000,5000,5000


In [13]:
# Determine the percentages of null values per column
#
((autolib_data.isnull() | autolib_data.isna()).sum() / autolib_data.index.size).round(3)

Address                0.000
Bluecar counter        0.000
Utilib counter         0.000
Utilib 1.4 counter     0.000
Charge Slots           0.000
Charging Status        0.000
City                   0.000
Displayed comment      0.978
ID                     0.000
Kind                   0.000
Geo point              0.000
Postal code            0.000
Public name            0.000
Rental status          0.000
Scheduled at           0.991
Slots                  0.000
Station type           0.000
Status                 0.000
Subscription status    0.000
year                   0.000
month                  0.000
day                    0.000
hour                   0.000
datetime               0.000
dtype: float64

There are two columns with missing values; 
- Displayed comment
- Scheduled at  

Displayed comment and Scheduled at columns had missing values over 95%. This will be previewed and a decision made whether to drop or maintain

In [14]:
# preview comments in the display comment column
#
autolib_data['Displayed comment'].describe()


count                                                111
unique                                                25
top       Station en parking (niv -2), accÃ¨s 4 av. Foch
freq                                                   8
Name: Displayed comment, dtype: object

In [15]:
# Dropping the display comment section
#
autolib_data.drop('Displayed comment', inplace = True, axis = 'columns')

The column is dropped since the data in it is in french can't be understood, and has no significance to the analysis

In [16]:
# previewing  comments in the scheduled at column
#
autolib_data['Scheduled at'].describe()

count                      47
unique                      8
top       2016-07-27T15:32:21
freq                       10
Name: Scheduled at, dtype: object

Data deals with schedules of the various electric cars. this column will be maintained since this data may be crucial during analysis

1.4.4) <font color="green">Duplicates</font>

Checking for duplicates

In [17]:
# Checking for duplicates
#
autolib_data.duplicated().axis = 1

No duplicates in dataframe

### 1.4.4) <font color="green">Uniformity</font>

Check column info, ordering the data appropiately, and introducing new data to improve the visualization of data

In [18]:
# Sorting data by date and station ID
#
autolib_df = autolib_data.sort_values(['ID', 'datetime'])

# Resetting the index of the newlly orderd data frame
#
autolib_df=autolib_df.reset_index()

# previewing dataset
#
autolib_df.head(2)

,index,Address,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,datetime
0,1352,61 quai Blanqui,0,0,0,0,nonexistent,Alfortville,alfortville-blanqui-61,STATION,"48.80188, 2.415843",94140,Alfortville/Blanqui/61,broken,NaN,0,station,closed,nonexistent,2018,4,2,19,2018-04-02 19:28:00
1,4528,61 quai Blanqui,0,0,0,0,nonexistent,Alfortville,alfortville-blanqui-61,STATION,"48.80188, 2.415843",94140,Alfortville/Blanqui/61,broken,NaN,0,station,closed,nonexistent,2018,4,4,9,2018-04-04 09:05:00


Reset index to factor in the new order the data frame has been ordered

In [19]:
# Specifying the variable values of rows and columns
#
rows = 5000
columns = 1

# Creating a list thet prints out 1, and 0 in alternating order
#

table = [[(i+j)%2 for i in range(columns)] for j in range(rows)]
# Creating a data frame from the table list
#
df = pd.DataFrame(table, columns = ['returned/picked'])

#  Converting the values  0 and 1 in the data frame to picked and return
#
df["returned/picked"].replace({ 0: "picked", 1: "returned"}, inplace=True)

# joining the two data frames using inner join
#
autolib = pd.concat([autolib_df, df], axis=1, join='inner')

# visualizing the dataframe
#
autolib.head()

,index,Address,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,datetime,returned/picked
0,1352,61 quai Blanqui,0,0,0,0,nonexistent,Alfortville,alfortville-blanqui-61,STATION,"48.80188, 2.415843",94140,Alfortville/Blanqui/61,broken,NaN,0,station,closed,nonexistent,2018,4,2,19,2018-04-02 19:28:00,picked
1,4528,61 quai Blanqui,0,0,0,0,nonexistent,Alfortville,alfortville-blanqui-61,STATION,"48.80188, 2.415843",94140,Alfortville/Blanqui/61,broken,NaN,0,station,closed,nonexistent,2018,4,4,9,2018-04-04 09:05:00,returned
2,4880,61 quai Blanqui,0,0,0,0,nonexistent,Alfortville,alfortville-blanqui-61,STATION,"48.80188, 2.415843",94140,Alfortville/Blanqui/61,broken,NaN,0,station,closed,nonexistent,2018,4,7,18,2018-04-07 18:40:00,picked
3,1693,61 quai Blanqui,0,0,0,0,nonexistent,Alfortville,alfortville-blanqui-61,STATION,"48.80188, 2.415843",94140,Alfortville/Blanqui/61,broken,NaN,0,station,closed,nonexistent,2018,4,9,9,2018-04-09 09:01:00,returned
4,4424,16 Rue Charles de Gaulle,2,0,0,0,nonexistent,Alfortville,alfortville-charlesdegaulle-16,STATION,"48.81456, 2.413135",94140,Alfortville/Charles de Gaulle/16,operational,NaN,0,station,ok,nonexistent,2018,4,1,0,2018-04-01 00:36:00,picked


- A new data frame is created with alternating values of picked and returned
- The newly created data frame is joined with the autolib_df after ordering hence creating alternating rows of columns and rows which will enable analysis of when the cars were returned or picked

In [20]:
# preview column info
#
autolib.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   index                5000 non-null   int64         
 1   Address              5000 non-null   object        
 2   Bluecar counter      5000 non-null   int64         
 3   Utilib counter       5000 non-null   int64         
 4   Utilib 1.4 counter   5000 non-null   int64         
 5   Charge Slots         5000 non-null   int64         
 6   Charging Status      5000 non-null   object        
 7   City                 5000 non-null   object        
 8   ID                   5000 non-null   object        
 9   Kind                 5000 non-null   object        
 10  Geo point            5000 non-null   object        
 11  Postal code          5000 non-null   int64         
 12  Public name          5000 non-null   object        
 13  Rental status        5000 non-nul

Data is uniform

## 1.5 Exporting datasheet



Downloading cleaned auto lib data data

In [21]:
# Exporting dataframe as CSV file
#
from google.colab import files
autolib.to_csv('autolib.csv')
files.download('autolib.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 1.6 Research question analysis

<font color="green">Main question analysis</font>

In [22]:
# Research Question
# Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018.
# Using loc method to isolate data for the month of april and year 2018
# Using value count method to determine the frequency of the various unique objects  in the hour column data set
# 
autolib['hour'].loc[(autolib['month'] == '4') & (autolib['year'] == '2018') & (autolib['returned/picked'] == 'picked') & (autolib['Bluecar counter']) & (autolib['City'] == 'Paris')].value_counts()

# Using mode method to determine the most popular hour for picking up shared electric blue car in the city of paris over the month of april 2018
#
autolib['hour'].loc[(autolib['month'] == '4') & (autolib['year'] == '2018') & (autolib['returned/picked'] == 'picked') & (autolib['Bluecar counter']) & (autolib['City'] == 'Paris')].mode()


0    12
dtype: object

The most popular hour of the day for picking up a shared electric blue car in the city of paris in the month of April 2018;
- Hour 12
- Frequency 34

<font color="green">Bonus question analysis</font>

In [23]:
# Challenge 1
# What is the most popular hour of returning cars
# Performing a value count of all unique hours using value count method
#
autolib['hour'].loc[(autolib['returned/picked'] == 'returned')].value_counts()

# determining most popular hour using mode method
#
autolib['hour'].loc[(autolib['returned/picked'] == 'returned')].mode()

0    21
dtype: object

The most popular time to return cars was 
- hour 21
- frequency 122

In [24]:
# Challenge 2
# Most popular station overall
# finding the value count of the most popular stations
#
autolib['Public name'].value_counts()

# determining most popular station using the mode method
autolib['Public name'].mode()

0    Paris/Porte de Montrouge/8
dtype: object

The most popular overall station;
- 'Paris/Porte de Montrouge/8'
- frequency of 13 

In [25]:
# Challenge 3
# Finding the most popular station at most picking popular hour
# finding most popular picking using mode method and frequecy using value counts method
#
autolib['hour'].loc[(autolib['returned/picked'] == 'picked')].value_counts()
autolib['hour'].loc[(autolib['returned/picked'] == 'picked')].mode()

# Getting the value counts of Public name using mode method at the most popular hour
#
autolib['Public name'].loc[(autolib['hour'] == '2')].value_counts()

# Determining the most popular station(public name) using mode method at the most popular hour
autolib['Public name'].loc[(autolib['hour'] == '2')].mode()


0            Courbevoie/Paix/2
1    Saint-Ouen/Louis Blanc/35
dtype: object

The most popular picking hour;
- hour 2
- frequency 119

Most popular stations at the most popular picking hour;
- Courbevoie/ Paix/2
- saintouen/louis Blanc/35
- freqeuncy 3 respectively

<font color="green">Blue Car</font>

In [26]:
# Challenge 4
#  postal code of the  most popular station
# Finding postal code of most popular station Paris/Porte de Montrouge/8 using location method to determine postal code
#
autolib['Postal code'].loc[(autolib['Public name'] == 'Paris/Porte de Montrouge/8')]

3635    75014
3636    75014
3637    75014
3638    75014
3639    75014
3640    75014
3641    75014
3642    75014
3643    75014
3644    75014
3645    75014
3646    75014
3647    75014
Name: Postal code, dtype: int64

Postal code of most popular station;
- postal code 75014

In [27]:
# Challenge 5
# Most popular postal code for picking blue cars
# Establishing the value counts of unique postal codes for blue cars using value count method
#
autolib['Postal code'].loc[(autolib['Bluecar counter']) & (autolib['returned/picked'] == 'picked')].value_counts()

# Determing the most popular postal code for blue cars using mode method
#
autolib['Postal code'].loc[(autolib['Bluecar counter']) & (autolib['returned/picked'] == 'picked')].mode()


0    75017
dtype: int64

The most popular postal code for picking blue cars was;
- postal code 75017 
- frequency of 59

The most popular station 'paris-portedemontrouge-8' does not belong to this postal code

In [28]:
# Challenge 6
# postal code most popular for  picking bluecars at the most popular picking hour
# using location to group the data sets by blue car and most popular hour, picked/returned 
# and using value count method to get order most popular postal code in this grouping
#
autolib['Postal code'].loc[(autolib['hour'] == '2') & (autolib['Bluecar counter']) & (autolib['returned/picked'] == 'picked')].value_counts()

# using location to group the data sets by blue car and most popular hour and 
#using mode method to get most popular postal code in this grouping
#
autolib['Postal code'].loc[(autolib['hour'] == '2') & (autolib['Bluecar counter']) & (autolib['returned/picked'] == 'picked')].mode()

0    75015
dtype: int64

The most popular postal codes for picking blue cars at the most popular hour were;
-     postal code 75015
-     frequency 4


Each with a frequency of two.

The most popular station 'Paris/Porte de Montrouge/8' does not belong to either of these postal codes

<font color="green">Utilib</font>

In [29]:
# Challenge 7
# Most popular postal code for picking Utilib cars overall
# selecting the location specifying the column for utilib cars. Perform a count of the post codes 
#
autolib['Postal code'].loc[(autolib['Utilib counter']) & (autolib['returned/picked'] == 'picked')].value_counts()

# Detrmining the postal code most used using the mode method
autolib['Postal code'].loc[(autolib['Utilib counter']) & (autolib['returned/picked'] == 'picked')].mode()

0    75017
dtype: int64

Most popular postal code for picking utilib cars;
- Postal code 75017
- Frequency 11

The postal code is similar to that of the most popular postal code for picking blue cars.However it has a lower frequency

The most popular station 'Paris/Porte de Montrouge/8'  does not belong to this postal code 

In [30]:
# Challenge 8
# postal code most popular for  picking Utilib cars at the most picking popular hour
# Finding the frequency of the unique postal codes at the most popular hour for utilib cars using value counts method
#
autolib['Postal code'].loc[(autolib['hour'] == '2') & (autolib['Utilib counter']) & (autolib['returned/picked'] == 'picked')].value_counts()

# using location to group the data sets by car (Utilib), most popular hour, and picking status. 
# Using mode function to get most popular postal code in this grouping
#
autolib['Postal code'].loc[(autolib['hour'] == '2') & (autolib['Utilib counter']) & (autolib['returned/picked'] == 'picked')].mode()



0    75015
1    75017
2    92310
3    94340
4    94800
dtype: int64

The most popular postal stations for picking utilib cars at the most popular picking our;
- 75015
- 75017
- 92310
- 94340
- 94800
- Frequency 1 

It shared a similar postal code '75015' with most popular postal code for picking blue cars at the most popular hour.However it had  a lower frequency

The most popular station 'Paris/Porte de Montrouge/8'  does not belong to either of these postal code 

<font color="green">Utilib 1.4</font>

In [31]:
# Challenge 9
# Most popular postal code for picking Utilib 1.4 cars overall
# selecting the location specifying the column for utilib cars. Perform a count of the post codes 
#
autolib['Postal code'].loc[(autolib['Utilib 1.4 counter']) & (autolib['returned/picked'] == 'picked')].value_counts()

# Detrmining the postal code most used using the mode method
autolib['Postal code'].loc[(autolib['Utilib 1.4 counter']) & (autolib['returned/picked'] == 'picked')].mode()


0    75017
dtype: int64

The most popular postal code for picking Utilib 1.4 cars overall;
- Postal code 75017 
- Frequency 19

Its postal code is similar to the most popular postal code for picking blue cars. However it has a lower frequency

In [32]:
# Challenge 10
# postal code most popular for  picking Utilib 1.4 cars at the most popular hour
# Finding the frequency of the unique postal codes at the most popular hour for utilib 1.4 cars using value counts method
#
autolib['Postal code'].loc[(autolib['hour'] == '2') & (autolib['Utilib 1.4 counter']) & (autolib['returned/picked'] == 'picked')].value_counts()

# using location to group the data sets by car (Utilib 1.4), most popular hour, and picking status. 
# Using mode function to get most popular postal code in this grouping
#
autolib['Postal code'].loc[(autolib['hour'] == '2') & (autolib['Utilib 1.4 counter']) & (autolib['returned/picked'] == 'picked')].mode()


0    92130
1    92150
dtype: int64

The most popular postal codes for picking utilib 1.4 cars at the most popular picking hour;
- Postal code 92130
- Postal code 92150
- Frequency 2 respectively

The most popular postal codes for picking utilib 1.4 cars were different from the most popular postal code for picking blue cars at this hour

The most popular station 'Paris/Porte de Montrouge/8'  does not belong to this postal code 

<font color="green">Other</font>

In [33]:
# Challenge 11
# Charging status
# establish number of charging status that are exitent 
# using value counts method to determine frequency and the unique characters in charging status column
#
autolib['Charging Status'].value_counts()

nonexistent    2886
operational    1995
broken          119
Name: Charging Status, dtype: int64

Charging status
- nonexistent    2886
- operational    1995
- broken          119

In [34]:
# Challenge 12
# Rental status
# Establish number of cars in operational condition, broken, planned and nonexitent
# Analysing the unique items in rental status column using value counts method
#
autolib['Rental status'].value_counts()

operational    4618
broken          332
future           47
nonexistent       3
Name: Rental status, dtype: int64

Rental status
- operational    4618
- broken          332
- future           47
- nonexistent       3